<a href="https://colab.research.google.com/github/mvince33/Coding-Dojo/blob/main/mock_belt_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/03 Week 3: Exploratory Viz/literacy_rates_updated.csv'
df = pd.read_csv(path)
df.head()

,Region,Country,Year,Age,Gender,Literacy rate
0,Central and Southern Asia,Afghanistan,2011,<15,female,0.1761206
1,Central and Southern Asia,Afghanistan,2011,<15,male,0.454171
2,NaN,Afghanistan,2011,15-24,female,0.3211322
3,NaN,Afghanistan,2011,15-24,male,0.6187907
4,Central and Southern Asia,Afghanistan,2011,25-64,female,0.0841276


In [ ]:
# Check for duplicates
df.duplicated().sum()
df[df.duplicated(keep = False)]
df.drop_duplicates(inplace = True)
df.duplicated().sum()

0

In [ ]:
# Check for missing values
df.isna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3303 entries, 0 to 3312
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Region         3299 non-null   object
 1   Country        3303 non-null   object
 2   Year           3303 non-null   object
 3   Age            3303 non-null   object
 4   Gender         3303 non-null   object
 5   Literacy rate  3298 non-null   object
dtypes: object(6)
memory usage: 180.6+ KB


In [ ]:
# Check the Region column.
df.loc[df['Region'].isna(), :]

# We can use the entry in 'Country' to determine
# what to use for 'Region' for missing values.
# All the missing entries in 'Region' should be
# 'Central and Southern Asia'
df['Region'] = df['Region'].fillna('Central and Southern Asia')
df['Region'].isna().sum()

0

In [ ]:
# Check the 'Literacy rate' column
print(df.loc[df['Literacy rate'].isna(), :])
df.loc[(df['Country'] == 'Spain') & 
       (df['Gender'] == 'female') &
       (df['Age'] != '65+'), :]

                             Region    Country  Year    Age  Gender  \
498  Eastern and South-Eastern Asia  Singapore  2010    65+    male   
608     Europe and Northern America    Albania  2012  15-24  female   
753     Europe and Northern America   Portugal  2011  15-24    male   
855     Europe and Northern America      Spain  2010    <15  female   
884     Europe and Northern America      Spain  2013  25-64    male   

    Literacy rate  
498           NaN  
608           NaN  
753           NaN  
855           NaN  
884           NaN  


,Region,Country,Year,Age,Gender,Literacy rate
855,Europe and Northern America,Spain,2010,<15,female,NaN
857,Europe and Northern America,Spain,2010,15-24,female,0.9968636
859,Europe and Northern America,Spain,2010,25-64,female,0.9881446
863,Europe and Northern America,Spain,2011,<15,female,0.9706557
865,Europe and Northern America,Spain,2011,15-24,female,0.9974597
867,Europe and Northern America,Spain,2011,25-64,female,0.9878686
871,Europe and Northern America,Spain,2012,<15,female,0.9721804
873,Europe and Northern America,Spain,2012,15-24,female,0.9970572
875,Europe and Northern America,Spain,2012,25-64,female,0.9891545
879,Europe and Northern America,Spain,2013,<15,female,0.9744054
